In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import psycopg2
from sqlalchemy import create_engine
import numpy as np
import csv
import boto
import time
import datetime as DT

""" Config """
REDSHIFTUSER = 
REDSHIFTPWD = 
AWSKEY=
AWSSECRETKEY=
SCHEMA = 
DATABASE = 
HOST = 
BUCKET = 
PROJECT = 
KEYNAME = ''
SKUSTRING = "('AZZ00147','DZZ85020','DZZ85763','DZZ86970','DZZ87371','DZZ87486','DZZ87518','DZZ90639','DZZ92404');"
##SKUSTRING = "('AZZ00147','DZZ85020','DZZ85763','DZZ86970','DZZ87371','DZZ87486','DZZ87518','DZZ90639','DZZ92404','DZZ92449','DZZ92926','DZZ93981','PZZ95782','PZZ95815','PZZ95886','PZZ96573','TZZ99870','TZZ99946','DZZ11112','DZZ86345','DZZ86407','DZZ87401','DZZ87827','DZZ89074','DZZ90335','DZZ91935','DZZ92830','DZZ94933','PZZ96012','PZZ96225','PZZ96299','PZZ96481','PZZ96611','AZZ01532','AZZ11496','DZZ85267','DZZ85878','DZZ86971','DZZ87381','DZZ91290','MZZ81507','MZZ96325','PZZ97215','AZZ01844','CZZ98889','DZZ85138','DZZ85883','DZZ87006','DZZ88977','DZZ89460','DZZ90673','DZZ91172','DZZ91263','MZZ81700','PZZ95726','PZZ95912','PZZ96000','PZZ97631','AZZ06437','AZZ19219','AZZ23121','DZZ87005','DZZ87360','DZZ87488','DZZ88274','DZZ97279','PZZ95727','PZZ96011','PZZ96293','AZZ00043','AZZ26617','AZZ35341','DZZ86246','DZZ86972','DZZ87252','DZZ89012','DZZ91562','DZZ92059','PZZ97267','PZZ97693','AZZ21063','AZZ22504','DZZ86245','DZZ86956','DZZ86962','DZZ87646','DZZ94234','DZZ94244','DZZ98575','MZZ83384','PZZ95870','PZZ96265','TZZ99935','AZZ00003','AZZ02218','AZZ04756','DZZ87372','DZZ87571','DZZ87647','DZZ87798','DZZ87898','DZZ89947','DZZ85267','DZZ87798','DZZ85763','DZZ87571','DZZ85138','DZZ94514','DZZ86407','DZZ90335','DZZ87486','DZZ89947');"



SyntaxError: invalid syntax (<ipython-input-2-4d5dc8232e4a>, line 12)

In [102]:
class PricingModel(object):

    def __init__(self):
        """ Connect to the database and fetch the Product Details.
        Two different libraries are used to connect to redshift. sqlalchemy is used for its integration with pandas
        while psycopg2 is used to write the results to the db.
        """
        self.conn = psycopg2.connect(
            host=HOST,
            user=REDSHIFTUSER, 
            port=5439, 
            password=REDSHIFTPWD,
            database=DATABASE)
        self.cur = self.conn.cursor() # create a cursor for executing queries

        engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" % (REDSHIFTUSER, REDSHIFTPWD, HOST, 5439, DATABASE)
        self.engine = create_engine(engine_string)

        ##query_string = "select * from boohoo.vw_product_details;"
        query_string = "select * from vw_product_details where sku in %s;" % (SKUSTRING)
        ##query_string = "select * from boohoo.vw_product_details where sku in ('AZZ00147','DZZ85020','DZZ85763','DZZ86970','DZZ87371','DZZ87486','DZZ87518','DZZ90639','DZZ92404','DZZ92449','DZZ92926');"
        self.df = pd.read_sql_query(query_string, self.engine)

        ## Changed the queries to eliminate the trycatch statements, changed the froup by clause in string 3 and made cosmetics changes
        q_string = """DROP TABLE IF EXISTS temp_table;
                        select split_part(orderlinesku, '-', 1) as sku , orderlineqty, orderdate, totalorderlinegrossvalue,orderlinepromotioncodes, orderlineid, bhline.orderid
                        into temp_table
                        from vw_order_line  bhline 
                        join vw_order_header  bhheader 
                        on bhline.orderid=bhheader.orderid;"""
        
        q_string2 = """DROP TABLE IF EXISTS temp_table2;
                        select * 
                        into temp_table2 
                        from temp_table tt 
                        LEFT JOIN bh_returnheader ret 
                        ON ret.ordernumber = tt.orderid 
                        WHERE ret.ordernumber IS NULL;"""
        q_string3 = """DROP TABLE IF EXISTS temp_table3;
                        select sku, sum(orderlineqty) number_orders, 
                        sum(totalorderlinegrossvalue) as total_cost,
                        pgdate_part('week', orderdate) as week, 
                        pgdate_part('year', orderdate) as year 
                        into temp_table3
                        from temp_table2
                        group by sku,pgdate_part('week', orderdate),pgdate_part('year', orderdate);"""
        
        self.cur.execute(q_string)
        print("q_string ran")
        self.conn.commit()
        self.cur.execute(q_string2)
        print("q_string2 ran")
        self.conn.commit()
        self.cur.execute(q_string3)
        print("q_string3 ran")
        self.conn.commit()

In [103]:
def cluster(self):
    """ Perform clustering on the textual features describing the items of clothing.
    """
    df = self.df.fillna('') #  change all nans to empty strings
    # add all of the text features together
    text_features = df['name'] + ' ' + df['product_group'] + ' ' + \
                    df['category'] + ' ' + df['department'] + ' ' + \
                    df['length'] + ' ' + df['origin'] + ' ' + \
                    df['style'] + ' ' + df['neckline'] + ' ' + \
                    df['trend'] + ' ' + df['heelshape'] + ' ' + \
                    df['toe'] + ' ' + df['sole'] + ' ' + \
                    df['heelheight'] + ' ' + df['wash'] + ' ' + \
                    df['enduse']

    numerical_features = df[['cost_price', 'weight']]

    tf = TfidfVectorizer(min_df=2)
    self.X = tf.fit_transform(text_features)

    self.nbrs = NearestNeighbors()
    self.nbrs.fit(self.X)




In [104]:
def calculate_prices(self):
## Added comments, made the code readable. Also changed the order of variable creation for readability
    # res_df_cl_all is only used to be passed as as an argument in the calculate_clearance_price function.
    # Intialized it here in order to avoid multiple calls to the DB. Not related to the other res_df in the fn.
    query_string1 = """SELECT si.ItemID,split_part(Code, '-', 1) as sku,Code,Price,sip.PriceBandId,pb.Name, bi.quantityonhand, si.manufacturer
                        FROM bh_StockItem si
                        join bh_StockItemPrice sip on si.ItemID = sip.ItemID
                        join bh_PriceBand pb on pb.PriceBandID = sip.PriceBandID
                        join bh_dwh_stock_levels bi on si.code = bi.productid
                        where pb.Name = 'Offer Price'
                        and split_part(Code, '-', 1) in %s""" % (SKUSTRING) 
                     
    res_df_cl_all = pd.read_sql_query(query_string1, self.engine) 
    print("Calc price Query 1 ran")
    query_string2 = """select sku , orderlineqty, orderdate
                        from temp_table
                        where sku in %s""" % (SKUSTRING) 
    orders_by_date = pd.read_sql_query(query_string2, self.engine) 
    print("Calc price Query 2 ran")
    orders_by_date['orderdate'] = pd.to_datetime(orders_by_date['orderdate'])
    
    
    with open(('%s.csv' %(PROJECT)), 'wb') as csvfile:
        self.spamwriter = csv.writer(csvfile)

        for index, row in self.df.iterrows():
            # [(k, v.sku) for k,v in self.df.iterrows() if v.sku == 'DZZ90335']
            ## Initialize Values
            published = self.df['published'][index]
            in_clearance = self.df['in_clearance'][index]
            act_weeks_on_sale = self.df['act_weeks_on_sale'][index]
            original_standard_price = self.df['original_standard_price'][index]
            current_standard_price = self.df['current_standard_price'][index]
            cost_price = self.df['cost_price'][index]
            sku = self.df['sku'][index]

            self.name = self.df['name'][index]
            self.department = self.df['product_group'][index]
            self.current_price = current_standard_price ## Why are we doing this??

            # Find Similar Skus
            res = self.nbrs.kneighbors(self.X[index], return_distance=False)
            print('Kmeans complete for the {0} time(s)'.format(index+1))
            similar_skus = tuple([self.df.sku[a] for a in res[0]])

            # Get orders of similar skus, summarized by week
            query_string = "select * from  temp_table3 where sku  in ('{0}','{1}','{2}','{3}','{4}')".format(similar_skus[0], similar_skus[1], similar_skus[2], similar_skus[3], similar_skus[4]);
            res_df = pd.read_sql_query(query_string, self.engine)               

            total_weeks = len(res_df.groupby(['week', 'year']).count())
            print(("{0} Weeks of orders").format(total_weeks))
            # Calculate some values of Similar skus
            try:
                ideal_price = sum(res_df['total_cost'])/sum(res_df['number_orders'])
            except:
                ideal_price = self.df['cost_price'][index] + 10
            ## merged the try except statements
            try:
                sim_items_sold_per_week = sum(res_df['number_orders'])/len(res_df)
                self.sim_items_sold_per_week = sum(res_df['number_orders'])/len(res_df)
            except:
                sim_items_sold_per_week = 10 
                self.sim_items_sold_per_week = 10
            
            # Calculate values of actual sku
            sp = res_df[res_df['sku'] == sku]
            try:
                items_sold_per_week = sum(sp['number_orders'])/len(sp)
            except:
                items_sold_per_week = sim_items_sold_per_week 

            #implement pricing logic
            new_price = (ideal_price/sim_items_sold_per_week)*items_sold_per_week
            weeks_on_sale = sp.count()['sku']

            q_string = """select totalorderlinegrossvalue from temp_table2 where orderlineid = (select max(orderlineid) from  temp_table where sku = '%s' and orderlinepromotioncodes = '')""" % sku 
            actual_df = pd.read_sql_query(q_string, self.engine)

            try:
                self.actual_price = actual_df['totalorderlinegrossvalue'].loc[0] 
            except:
                self.actual_price = 0

            try:
                ss = sp.loc[0]
                self.current_price = ss['total_cost']/ss['number_orders'] # practically, is this different from actual price?
            except Exception as e: 
                self.current_price = self.actual_price
            if self.current_price == 0:
                self.current_price = current_standard_price 

            # Introduce some rule based checks
            min_price = 0
            if act_weeks_on_sale < 5:
                min_price = price_point(self, self.df['product_group'][index].upper(), self.current_price)

            if new_price < min_price:
                new_price = min_price
            if new_price > (original_standard_price + original_standard_price/10):
                new_price = (original_standard_price + original_standard_price/10)
            
            print(sku)
            res_df_cl = res_df_cl_all[res_df_cl_all['sku']==sku]    # to pass to the function below
         
            calculate_clearance_price(self,sku, res_df_cl, items_sold_per_week,orders_by_date, new_price, ideal_price, sim_items_sold_per_week, cost_price, published, in_clearance, act_weeks_on_sale, original_standard_price, current_standard_price)
            
            ## Ideal price need not be passed



In [105]:
def calculate_clearance_price(self, sku, res_df_cl, items_sold_per_week,orders_by_date, new_price, ideal_price, sim_items_sold_per_week, cost_price, published, in_clearance, act_weeks_on_sale, original_standard_price, current_standard_price):
    ## Ideal price is not used
    try:
        price = res_df_cl['price'].loc[0]
    except: 
        price = 0
    clearance_price = price
    output_list = {}
    predicted_throughput = self.sim_items_sold_per_week 
    stock_level = sum(res_df_cl['quantityonhand'])
    try:
        weeks_of_stock = stock_level/items_sold_per_week
    except: 
        weeks_of_stock = 0
    #old calc current_velocity = items_sold_per_week ## What's the purpose of this?? Can be removed
    # Calculate the number of items sold in the last 7 days
     try:
        current_velocity = sum(orders_by_date[(orders_by_date.sku==sku) &
                                                     (pd.to_datetime(orders_by_date['orderdate'])> (DT.date.today() - DT.timedelta(days = 7))) &
                                                     (pd.to_datetime(orders_by_date['orderdate']) <= DT.date.today())].orderlineqty)
    except:
        current_velocity = 0
        
    current_margin = self.current_price - cost_price
    for weeks in [2,4,6,8,10]:
        if stock_level < (weeks*items_sold_per_week):
            clearance_price = new_price
            tp = predicted_throughput 
        else:
            try:
                clearance_price = new_price*(items_sold_per_week/(stock_level/weeks))
                tp = (predicted_throughput - predicted_throughput * (items_sold_per_week/(stock_level/weeks))) + predicted_throughput 
            except: 
                clearance_price = new_price
                tp = predicted_throughput 

        if clearance_price < cost_price:
            clearance_price = cost_price
        if new_price < price:
            new_price = price                   
        output_list["week{0}".format(weeks)] = clearance_price
        output_list["margin{0}".format(weeks)] = clearance_price - cost_price
        output_list["tp".format(weeks)] = tp
        output_list["cd{0}".format(weeks)] = (clearance_price - self.current_price)*stock_level
        #print(("new price is {0}").format(new_price))
    cost_decision = (new_price - self.current_price)*stock_level
    predicted_margin = new_price - cost_price
    other_outputvars = dict(sku = sku, new_price = new_price, current_price = self.current_price, actual_price = self.actual_price,weeks_of_stock = weeks_of_stock ,current_velocity = current_velocity,current_margin = current_margin,predicted_throughput = predicted_throughput,predicted_margin = predicted_margin, stock_level = stock_level, name = self.name, department = self.department,cost_decision = cost_decision)
    output_list.update(other_outputvars)
    self.spamwriter.writerow([output_list])
    #self.spamwriter.writerow([sku, new_price, week2, week4, week6, week8, week10, self.current_price,self.actual_price, weeks_of_stock , current_velocity, current_margin,predicted_throughput,predicted_margin, stock_level, self.name, self.department, margin2, margin4, margin6, margin8, margin10, tp2, tp4, tp6, tp8, tp10, cd2, cd4, cd6, cd8, cd10, cost_decision ])

In [106]:
  def price_point(self, group, current_price): ## What is the function supposed to do?
        query = """select * from price_architecture where sub_group = '{0}' order by price""".format(group)
        price_points = pd.read_sql_query(query, self.engine)
        prices = price_points['price']
        try:
            idx = np.where((np.abs(prices.tolist())-current_price)==((np.abs(prices.tolist())-current_price).argmin()))[0][0]
        except:
            return 0
        idx = idx - 2
        if idx < 0:
            idx = 0
        return price_points['price'][idx]

In [107]:
 def write_to_db(self):
    # removed try catch from sql and made some cosmetic changes.
    # default delimiter is tab, added correct delimiter and also added removequotes.
    self.cur.execute("DROP TABLE IF EXISTS temp_table;")
    self.conn.commit()

    self.cur.execute("DROP TABLE IF EXISTS temp_table2;")
    self.conn.commit()

    self.cur.execute("DROP TABLE IF EXISTS temp_table3;")
    self.conn.commit()


    s3 = boto.connect_s3(AWSKEY, AWSSECRETKEY)
    bucket = s3.get_bucket(BUCKET)
    key_name =  '%s-%s' % (PROJECT,int(time.time()))
    print(key_name)
    key = bucket.new_key(key_name)
    key.set_contents_from_filename('%s.csv' %(PROJECT))

    self.cur.execute("DROP TABLE IF EXISTS boohoo.boohoo_pricing;")
    self.conn.commit()

    self.cur.execute("create table pricing (sku varchar,suggested_price float ,week2_price float , week4_price float, week6_price float, week8_price float, week10_price float, current_price float, actual_price float, weeks_of_stock float, Current_weekly_velocity float, current_margin float, predicted_throughput float, predicted_margin float, stock_level float, name varchar, department varchar, week2_margin float, week4_margin float, week6_margin float, week8_margin float, week10_margin float, predicted_throughput_was_relevant_to_2_weeks float, predicted_throughput_was_relevant_to_4_weeks float, predicted_throughput_was_relevant_to_6_weeks float, predicted_throughput_was_relevant_to_8_weeks float, predicted_throughput_was_relevant_to_10_weeks float);")
    self.conn.commit()
    self.cur.execute("copy pricing from 's3://%s/%s' CSV credentials 'aws_access_key_id=%s;aws_secret_access_key=%s';" % (BUCKET,key_name, AWSKEY, AWSSECRETKEY),removequotes,delimiter ',')
    self.conn.commit()


Functional Calls

In [108]:
pm = PricingModel()

q_string ran
q_string2 ran
q_string3 ran


In [109]:
cluster(pm)

In [110]:
calculate_prices(pm)

Calc price Query 1 ran
Calc price Query 2 ran
Kmeans complete for the 1 time(s)
9 Weeks of orders
AZZ00147
Kmeans complete for the 2 time(s)
17 Weeks of orders
DZZ85763
Kmeans complete for the 3 time(s)
9 Weeks of orders
DZZ86970
Kmeans complete for the 4 time(s)
17 Weeks of orders
DZZ90639
Kmeans complete for the 5 time(s)
9 Weeks of orders
DZZ87486
Kmeans complete for the 6 time(s)
8 Weeks of orders
DZZ87518
Kmeans complete for the 7 time(s)
17 Weeks of orders
DZZ92404
Kmeans complete for the 8 time(s)
17 Weeks of orders
DZZ85020
Kmeans complete for the 9 time(s)
9 Weeks of orders
DZZ87371


In [111]:
write_to_db(pm)

boohoo-pricing-1456495025


InternalError: Load into table 'boohoo_pricing' failed.  Check 'stl_load_errors' system table for details.
